In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from scipy.stats import uniform
from scipy.stats import randint
import cv2

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from extract_data import extract_data

np.random.seed(52384)

In [2]:
def pca_reduce(x_train, x_test, num_components):
  # Standardize the data
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  # Perform PCA
  pca = PCA(n_components=num_components)
  X_train_pca = pca.fit_transform(X_train_scaled)
  X_test_pca = pca.transform(X_test_scaled)
  return X_train_pca, X_test_pca

## 2. Two subject

In [3]:
# extract train and test data
x_train, y_train, x_test, y_test = extract_data('../ds000105_R2.0.2', sub_num=2)
x_train2 = x_train.reshape(40*64*x_train.shape[2], x_train.shape[3])
x_test2 = x_test.reshape(40*64*x_train.shape[2], x_test.shape[3])
x_train2 = x_train2.T
x_test2 = x_test2.T
mean_train = np.mean(x_train2, axis=0)
std_train = np.std(x_train2, axis=0)
x_train_norm = (x_train2 - mean_train) / std_train
x_test_norm = (x_test2 - mean_train) / std_train

In [4]:
# denoise: Gaussian smoothing
x_train_gs = cv2.GaussianBlur(x_train_norm, (5, 5), 0)
x_test_gs = cv2.GaussianBlur(x_test_norm, (5, 5), 0)
# PCA reduce order
# x_train_pca, x_test_pca=pca_reduce(x_train_gs,x_test_gs,150)
x_train_pca, x_test_pca=pca_reduce(x_train_norm,x_test_norm,150)

In [5]:
svm = SVC(random_state=42)
hyper_param_svm = {
    'C': np.logspace(-3, 5, 9),
    'kernel': ['poly','rbf'],
    'degree': [1, 2, 3, 4],
    'gamma': ['scale', 'auto'] + list(np.logspace(-8, 3, 12)),
}
model_cv_svm = RandomizedSearchCV(svm,
                        param_distributions = hyper_param_svm,
                        n_iter = 100,
                        scoring = 'accuracy',
                        cv = 10, 
                        n_jobs = -1)

In [6]:
# rf = RandomForestClassifier(random_state=42)
# hyper_param_rf = {
#     'n_estimators': randint(50, 500),
#     'max_depth': [None] + list(range(5, 30, 5)),
#     'min_samples_split': randint(2, 20),
#     'min_samples_leaf': randint(1, 20),
#     'max_features': ['sqrt', 'log2'] + [x_train_gs.shape[1]],
#     'criterion': ['gini', 'entropy'],
#     'bootstrap': [True, False]
# }
# model_cv_rf = RandomizedSearchCV(rf, 
#                         param_distributions = hyper_param_rf,
#                         n_iter = 100,
#                         scoring = 'accuracy',
#                         cv = 10, 
#                         n_jobs = -1)

In [7]:
knn = KNeighborsClassifier()
hyper_param_knn = {'n_neighbors': range(1,150,10),
                   'weights': ['uniform', 'distance']}
model_cv_knn = GridSearchCV(estimator=knn,
                        param_grid = hyper_param_knn,
                        scoring = 'accuracy', 
                        cv = 10, 
                        n_jobs = -1)

In [8]:
lr = LogisticRegression(penalty='l2', solver='liblinear', random_state=42)
hyper_param_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],}
model_cv_lr = GridSearchCV(estimator=lr,
                        param_grid = hyper_param_lr,
                        scoring = 'accuracy', 
                        cv = 10, 
                        n_jobs = -1)

### 2.1 Gaussian smoothing

#### 2.1.1 SVM

In [9]:
search1 = model_cv_svm.fit(x_train_gs, y_train)
search1.best_params_

{'kernel': 'poly', 'gamma': 0.0001, 'degree': 1, 'C': 10000.0}

In [10]:
print('The best average accuracy over CV is:')
print(search1.cv_results_['mean_test_score'][search1.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.5992424242424242


In [11]:
model = SVC(kernel=search1.best_params_.get('kernel'), gamma=search1.best_params_.get('gamma'), 
            C=search1.best_params_.get('C'), degree=search1.best_params_.get('degree'), random_state=42)
model.fit(x_train_gs, y_train)

SVC(C=10000.0, degree=1, gamma=0.0001, kernel='poly', random_state=42)

In [12]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6728971962616822
              precision    recall  f1-score   support

         0.0       0.65      0.76      0.70        54
         1.0       0.70      0.58      0.64        53

    accuracy                           0.67       107
   macro avg       0.68      0.67      0.67       107
weighted avg       0.68      0.67      0.67       107



#### 2.1.2 Random Forest

In [13]:
# search2 = model_cv_rf.fit(x_train_gs, y_train)
# search2.best_params_

In [14]:
# print('The best average accuracy over CV is:')
# print(search2.cv_results_['mean_test_score'][search2.cv_results_['rank_test_score']==1][0])

In [15]:
# model = RandomForestClassifier(n_estimators=search2.best_params_.get('n_estimators'), 
#                                max_depth=search2.best_params_.get('max_depth'), 
#                                min_samples_split=search2.best_params_.get('min_samples_split'), 
#                                min_samples_leaf=search2.best_params_.get('min_samples_leaf'),
#                                max_features=search2.best_params_.get('max_features'),
#                                criterion=search2.best_params_.get('criterion'),
#                                bootstrap=search2.best_params_.get('bootstrap'))
# model.fit(x_train_gs, y_train)

In [16]:
# y_pred = model.predict(x_test_gs)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

#### 2.1.3 KNN

In [17]:
search3 = model_cv_knn.fit(x_train_gs, y_train)
search3.best_params_

{'n_neighbors': 131, 'weights': 'uniform'}

In [18]:
print('The best average accuracy over CV is:')
print(search3.cv_results_['mean_test_score'][search3.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.4924242424242424


In [19]:
model = KNeighborsClassifier(n_neighbors=search3.best_params_.get('n_neighbors'),
                           weights=search3.best_params_.get('weights'))
model.fit(x_train_gs, y_train)

KNeighborsClassifier(n_neighbors=131)

In [20]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5046728971962616
              precision    recall  f1-score   support

         0.0       0.51      0.35      0.42        54
         1.0       0.50      0.66      0.57        53

    accuracy                           0.50       107
   macro avg       0.51      0.51      0.49       107
weighted avg       0.51      0.50      0.49       107



#### 2.1.4 Logistic Regression

In [21]:
search10 = model_cv_lr.fit(x_train_gs, y_train)
search10.best_params_

{'C': 100}

In [22]:
print('The best average accuracy over CV is:')
print(search10.cv_results_['mean_test_score'][search10.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.5817234848484849


In [23]:
model = LogisticRegression(C=search10.best_params_.get('C'), penalty='l2', solver='liblinear', random_state=42)
model.fit(x_train_gs, y_train)

LogisticRegression(C=100, random_state=42, solver='liblinear')

In [24]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6728971962616822
              precision    recall  f1-score   support

         0.0       0.65      0.78      0.71        54
         1.0       0.71      0.57      0.63        53

    accuracy                           0.67       107
   macro avg       0.68      0.67      0.67       107
weighted avg       0.68      0.67      0.67       107



### 2.2 PCA

#### 2.2.1 SVM

In [25]:
search4 = model_cv_svm.fit(x_train_pca, y_train)
search4.best_params_

{'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'C': 100.0}

In [26]:
print('The best average accuracy over CV is:')
print(search4.cv_results_['mean_test_score'][search4.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.9106060606060605


In [27]:
model = SVC(kernel=search4.best_params_.get('kernel'), gamma=search4.best_params_.get('gamma'), 
            C=search4.best_params_.get('C'), degree=search4.best_params_.get('degree'), random_state=42)
model.fit(x_train_pca, y_train)

SVC(C=100.0, degree=4, random_state=42)

In [28]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9252336448598131
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93        54
         1.0       0.94      0.91      0.92        53

    accuracy                           0.93       107
   macro avg       0.93      0.93      0.93       107
weighted avg       0.93      0.93      0.93       107



#### 2.2.2 Random Forest

In [29]:
# search5 = model_cv_rf.fit(x_train_pca, y_train)
# search5.best_params_

In [30]:
# print('The best average accuracy over CV is:')
# print(search5.cv_results_['mean_test_score'][search5.cv_results_['rank_test_score']==1][0])

In [31]:
# model = RandomForestClassifier(n_estimators=search5.best_params_.get('n_estimators'), 
#                                max_depth=search5.best_params_.get('max_depth'), 
#                                min_samples_split=search5.best_params_.get('min_samples_split'), 
#                                min_samples_leaf=search5.best_params_.get('min_samples_leaf'),
#                                max_features=search5.best_params_.get('max_features'),
#                                criterion=search5.best_params_.get('criterion'),
#                                bootstrap=search5.best_params_.get('bootstrap'))
# model.fit(x_train_pca, y_train)

In [32]:
# y_pred = model.predict(x_test_pca)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

#### 2.2.3 KNN

In [33]:
search6 = model_cv_knn.fit(x_train_pca, y_train)
search6.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [34]:
print('The best average accuracy over CV is:')
print(search6.cv_results_['mean_test_score'][search6.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.8582386363636363


In [35]:
model = KNeighborsClassifier(n_neighbors=search6.best_params_.get('n_neighbors'),
                           weights=search6.best_params_.get('weights'))
model.fit(x_train_pca, y_train)

KNeighborsClassifier(n_neighbors=1)

In [36]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8411214953271028
              precision    recall  f1-score   support

         0.0       0.80      0.91      0.85        54
         1.0       0.89      0.77      0.83        53

    accuracy                           0.84       107
   macro avg       0.85      0.84      0.84       107
weighted avg       0.85      0.84      0.84       107



#### 2.2.4 Logistic Regression

In [37]:
search11 = model_cv_lr.fit(x_train_pca, y_train)
search11.best_params_

{'C': 0.1}

In [38]:
print('The best average accuracy over CV is:')
print(search11.cv_results_['mean_test_score'][search11.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.8369318181818182


In [39]:
model = LogisticRegression(C=search11.best_params_.get('C'), penalty='l2', solver='liblinear', random_state=42)
model.fit(x_train_pca, y_train)

LogisticRegression(C=0.1, random_state=42, solver='liblinear')

In [40]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8785046728971962
              precision    recall  f1-score   support

         0.0       0.85      0.93      0.88        54
         1.0       0.92      0.83      0.87        53

    accuracy                           0.88       107
   macro avg       0.88      0.88      0.88       107
weighted avg       0.88      0.88      0.88       107



### 2.3 No smoothing and PCA

#### 2.3.1 SVM

In [41]:
search7 = model_cv_svm.fit(x_train_norm, y_train)
search7.best_params_

{'kernel': 'poly', 'gamma': 0.01, 'degree': 4, 'C': 10.0}

In [42]:
print('The best average accuracy over CV is:')
print(search7.cv_results_['mean_test_score'][search7.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.9108901515151515


In [43]:
model = SVC(kernel=search7.best_params_.get('kernel'), gamma=search7.best_params_.get('gamma'), 
            C=search7.best_params_.get('C'), degree=search7.best_params_.get('degree'), random_state=42)
model.fit(x_train_norm, y_train)

SVC(C=10.0, degree=4, gamma=0.01, kernel='poly', random_state=42)

In [44]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9065420560747663
              precision    recall  f1-score   support

         0.0       0.94      0.87      0.90        54
         1.0       0.88      0.94      0.91        53

    accuracy                           0.91       107
   macro avg       0.91      0.91      0.91       107
weighted avg       0.91      0.91      0.91       107



#### 2.3.2 Random Forest

In [45]:
# search8 = model_cv_rf.fit(x_train_norm, y_train)
# search8.best_params_

In [46]:
# print('The best average accuracy over CV is:')
# print(search8.cv_results_['mean_test_score'][search8.cv_results_['rank_test_score']==1][0])

In [47]:
# model = RandomForestClassifier(n_estimators=search8.best_params_.get('n_estimators'), 
#                                max_depth=search8.best_params_.get('max_depth'), 
#                                min_samples_split=search8.best_params_.get('min_samples_split'), 
#                                min_samples_leaf=search8.best_params_.get('min_samples_leaf'),
#                                max_features=search8.best_params_.get('max_features'),
#                                criterion=search8.best_params_.get('criterion'),
#                                bootstrap=search8.best_params_.get('bootstrap'))
# model.fit(x_train_norm, y_train)

In [48]:
# y_pred = model.predict(x_test_norm)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

#### 2.3.3 KNN

In [49]:
search9 = model_cv_knn.fit(x_train_norm, y_train)
search9.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [50]:
print('The best average accuracy over CV is:')
print(search9.cv_results_['mean_test_score'][search9.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.8675189393939394


In [51]:
model = KNeighborsClassifier(n_neighbors=search9.best_params_.get('n_neighbors'),
                           weights=search9.best_params_.get('weights'))
model.fit(x_train_norm, y_train)

KNeighborsClassifier(n_neighbors=1)

In [52]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9252336448598131
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93        54
         1.0       0.94      0.91      0.92        53

    accuracy                           0.93       107
   macro avg       0.93      0.93      0.93       107
weighted avg       0.93      0.93      0.93       107



#### 2.3.4 Logistic Regression

In [53]:
search12 = model_cv_lr.fit(x_train_norm, y_train)
search12.best_params_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'C': 1000}

In [54]:
print('The best average accuracy over CV is:')
print(search12.cv_results_['mean_test_score'][search12.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.8652462121212121


In [55]:
model = LogisticRegression(C=search12.best_params_.get('C'), penalty='l2', solver='liblinear', random_state=42)
model.fit(x_train_norm, y_train)

LogisticRegression(C=1000, random_state=42, solver='liblinear')

In [56]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9065420560747663
              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91        54
         1.0       0.92      0.89      0.90        53

    accuracy                           0.91       107
   macro avg       0.91      0.91      0.91       107
weighted avg       0.91      0.91      0.91       107

